In [18]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [20]:
from data import dataloader
import pandas as pd
import numpy as np

In [11]:
pilot_data = dataloader.break_out_blocks(
    dataloader.load_from_firebase()
)

Data loaded from Firebase!


In [21]:
for target in list(pilot_data.target.unique()):
    ib =max(
        pilot_data.loc[
            pilot_data.target==target
        ].iteration_block.dropna()
    )
    print(": ".join([
        target,
        "{0} on-target samples in {1} iteration block{2}".format(
            str(len(pilot_data.loc[
                (pilot_data.target == target) &
                (pilot_data.ontarget)
            ])),
            "%.0f" % ib,
            "s" if ib != 1 else ""
        )
    ]))


none: 19 on-target samples in 2 iteration blocks
environment: 106 on-target samples in 4 iteration blocks
body: 101 on-target samples in 2 iteration blocks
cup: 33 on-target samples in 3 iteration blocks
food: 35 on-target samples in 4 iteration blocks
nails: 69 on-target samples in 3 iteration blocks
smoke: 92 on-target samples in 3 iteration blocks
eyebrow: 235 on-target samples in 5 iteration blocks
nose: 249 on-target samples in 4 iteration blocks
above-ear: 403 on-target samples in 5 iteration blocks
behind-ear: 220 on-target samples in 4 iteration blocks
opposite-cheek: 74 on-target samples in 6 iteration blocks
chin: 166 on-target samples in 1 iteration block
cheek: 686 on-target samples in 6 iteration blocks
forehead: 176 on-target samples in 1 iteration block
top-head: 160 on-target samples in 1 iteration block
back-head: 170 on-target samples in 1 iteration block
opposite-face: 381 on-target samples in 2 iteration blocks
paint-mouth: 81 on-target samples in 3 iteration blocks

In [45]:
pilot_subset = []
for row in pilot_data[(pilot_data.target == 'cheek') & (pilot_data.ontarget)][['thermopile1','thermopile2','thermopile3','thermopile4']].values.tolist():
    pilot_subset.append({'in':row, 'out':True})
pilot_subset

[{'in': ['79.1', '83.9', '84.4', '83.0'], 'out': True},
 {'in': ['78.6', '84.0', '84.5', '83.1'], 'out': True},
 {'in': ['78.4', '84.0', '84.5', '83.1'], 'out': True},
 {'in': ['83.8', '83.9', '80.3', '83.4'], 'out': True},
 {'in': ['83.8', '83.1', '81.4', '84.0'], 'out': True},
 {'in': ['83.3', '83.4', '80.9', '83.6'], 'out': True},
 {'in': ['83.1', '84.1', '80.4', '83.3'], 'out': True},
 {'in': ['80.8', '82.8', '80.6', '83.3'], 'out': True},
 {'in': ['83.1', '83.3', '80.6', '83.3'], 'out': True},
 {'in': ['83.1', '83.5', '81.4', '83.6'], 'out': True},
 {'in': ['82.5', '83.5', '80.4', '83.3'], 'out': True},
 {'in': ['82.3', '82.8', '80.6', '83.3'], 'out': True},
 {'in': ['83.5', '83.6', '80.9', '83.3'], 'out': True},
 {'in': ['81.0', '82.9', '80.9', '83.6'], 'out': True},
 {'in': ['81.0', '83.0', '80.6', '83.5'], 'out': True},
 {'in': ['83.6', '83.8', '81.3', '82.9'], 'out': True},
 {'in': ['83.3', '83.8', '80.8', '83.5'], 'out': True},
 {'in': ['80.8', '82.8', '80.5', '83.5'], 'out':